<a href="https://colab.research.google.com/github/DipankarJDutta/WBC-Biomarker-ID-for-FASD-GDM-/blob/Optimal-Network-Weight-Initialization-for-MLP-Model/MLP_T_cells_AS_EtOH_GDM_z_score_network%20weight%20initialization_gridsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Loading essentials for batch size (Source: scikit-learn grid search)
from numpy import loadtxt
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [0]:
# Fixing random seed for reproducibility
from numpy.random import seed
seed(7)
from tensorflow import set_random_seed
set_random_seed(7)

In [0]:
#Creating model required for KerasClassifier
def create_model(init_mode='uniform'):
    #create model
    model = Sequential()
    model.add(Dense(12, input_dim=25, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(8, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
    #compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [0]:
#load z-scored inclevel values of significantly AS genes in T-cells common to EtOH & GDM datasets
dataset = loadtxt ('z-score_T-cell.csv', delimiter = ',')

In [0]:
#split dataset into input and output variables (25 input, 1 output)
x = dataset [:, 0:25]
y = dataset [:, 25]

In [0]:
#Create model for gridsearch
model = KerasClassifier (build_fn=create_model, batch_size = 21, epochs = 100, verbose=0)

In [7]:
#Define Grid Search parameters for optimal model wight initializer for input T-cell dataset with 47 samples, epoch size of 100 and optimal batch size of 21)
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
#Jobs are run in parallel with n_jobs=-1; 5-fold cross-validation
grid = GridSearchCV(estimator=model, param_grid = param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(x, y)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [8]:
#Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.666667 using {'init_mode': 'glorot_uniform'}
0.625000 (0.128650) with: {'init_mode': 'uniform'}
0.500000 (0.120761) with: {'init_mode': 'lecun_uniform'}
0.541667 (0.128650) with: {'init_mode': 'normal'}
0.583333 (0.168050) with: {'init_mode': 'zero'}
0.520833 (0.125577) with: {'init_mode': 'glorot_normal'}
0.666667 (0.054857) with: {'init_mode': 'glorot_uniform'}
0.541667 (0.200981) with: {'init_mode': 'he_normal'}
0.479167 (0.125577) with: {'init_mode': 'he_uniform'}
